# Download ORACC JSON Files
This script downloads open data from the Open Richly Annotated Cuneiform Corpus ([ORACC](http://oracc.org)) in `json` format. The JSON files are made available in a ZIP file. For a description of the various JSON files included in the ZIP see the [open data](http://oracc.museum.upenn.edu/doc/opendata) page on [ORACC](http://oracc.org). 

In [1]:
import pandas as pd   
import requests
import io
import tqdm
import json
import os
import zipfile
import errno
import time

# Create Download Directory
Create a directory called `jsonzip`. If the directory already exists, do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
ROOT_PATH = os.getcwd()

PROJECTS_METADATA_PATH = os.path.join('projectsmetadata')

CSV_PROJECTS_DF = os.path.join(PROJECTS_METADATA_PATH, 'projects.csv')
LIST_OF_PROJECTS = os.path.join(PROJECTS_METADATA_PATH, 'projects.txt')

ZIP_PATH = os.path.join(os.getcwd(), 'jsonzip')
EXTRACT_PATH = os.path.join(os.getcwd(), 'projectsdata')

In [3]:
try:
    os.mkdir(ZIP_PATH)
except OSError as exc:
    if exc.errno !=errno.EEXIST:
        raise
    pass

try:
    os.mkdir(EXTRACT_PATH)
except OSError as exc:
    if exc.errno !=errno.EEXIST:
        raise
    pass

try:
    os.mkdir(PROJECTS_METADATA_PATH)
except OSError as exc:
    if exc.errno !=errno.EEXIST:
        raise
    pass

# Get up-to-date list of existing projects and subprojects

As listed in [The Oracc Project List](https://oracc.museum.upenn.edu/projectlist.html)

In [4]:
projects_url = 'https://oracc.museum.upenn.edu/projectlist.html'
response = requests.get(projects_url, verify=False)

response_data = response.text

lines_in_html = response_data.split('\n')

projects_dict = {}
run_shortcuts = []

idx=0
for line in lines_in_html:
    if 'href="./' in line:
        line_parts = line.split('href="./')
        line_parts_2 = line_parts[1].split('">')
        project_shortcut = line_parts_2[0]
        project_shortcut = project_shortcut.replace('/', '-')
        if project_shortcut in run_shortcuts:
            continue
        else:
            line_parts_3 = line_parts_2[1].split('</a>')
            project_name = line_parts_3[0]
            projects_dict[idx] = {'name': project_name, 'shortcut': project_shortcut, 'project_json_link': f'https://oracc.museum.upenn.edu/json/{project_shortcut}.zip'}
            run_shortcuts.append(project_shortcut)
            idx += 1
            
print('Up-to-date list of projects has been created.')

# Extracting projects to csv file projects.csv
projects_df = pd.DataFrame.from_dict(projects_dict)
projects_df = projects_df.transpose()
projects_df.to_csv(CSV_PROJECTS_DF)
print(f'File projects.csv has been saved to {CSV_PROJECTS_DF}.')

# Extracting list of projects to txt file projects.txt
with open(LIST_OF_PROJECTS, 'w', encoding='utf-8') as txt_file:
    txt_file.write('\n'.join(run_shortcuts))
print(f'File projects.txt has been saved to {LIST_OF_PROJECTS}.')

c:\Users\valek\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oracc.museum.upenn.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Up-to-date list of projects has been created.
File projects.csv has been saved to projectsmetadata\projects.csv.
File projects.txt has been saved to projectsmetadata\projects.txt.


# Parse the file with text IDs
The following code reads the file with text ID and pulls out the project names. The code removes accidental spaces at the beginning and the end of each line as well as blank lines. Each line in the file with text IDs is split at the first space - everything after the first space is ignored. The last 8 digits of the text ID are removed, to leave only the project name.

#FV note: I want only projects, not texts --> iggnoring the removal of last 8 characters.

In [5]:
with open(LIST_OF_PROJECTS, 'r') as f:
    projects = f.read().split('\n')

In [6]:
for project in projects:
    print(project)

adsd
adsd-adart1
adsd-adart2
adsd-adart3
adsd-adart5
adsd-adart6
aemw
aemw-alalakh-idrimi
aemw-amarna
akklove
amgg
ario
armep
arrim
asbp
asbp-ninmed
asbp-rlasb
atae
atae-assur
atae-burmarina
atae-durkatlimmu
atae-durszarrukin
atae-guzana
atae-huzirina
atae-imgurenlil
atae-kalhu
atae-kunalia
atae-mallanate
atae-marqasu
atae-nineveh
atae-samal
atae-szibaniba
atae-tilbarsip
atae-tuszhan
babcity
balt
blms
borsippa
btmao
btto
cams-akno
cams-barutu
cams-gkab
cams-ludlul
cams-selbi
cams-tlab
cdli
ckst
cmawro
cmawro-cmawr1
cmawro-cmawr2
cmawro-cmawr3
cmawro-maqlu
contrib-amarna
contrib-jacobsen
contrib-lambert
ctij
dcclt
dcclt-ebla
dcclt-jena
dcclt-nineveh
dcclt-signlists
dccmt
dsst
ecut
edlex
eisl
epsd2
etcsri
glass
hbtin
iraq
iraq-iraq85
kish
kish-fieldmus
kish-fieldmus-fmod
kish-mathaffield
nere
nimrud
obel
obmc
obta
ogsl
oimea
osl
pnao
qcat
riao
ribo
ribo-bab7scores
ribo-babylon10
ribo-babylon2
ribo-babylon3
ribo-babylon4
ribo-babylon5
ribo-babylon6
ribo-babylon7
ribo-babylon8
ribo-sources

## Download the ZIP files
For each project from which files are to be processed download the entire project (all the json files) from `http://build-oracc.museum.upenn.edu/json/`. The file is called `PROJECT.zip` (for instance: `dcclt.zip`). For subprojects the file is called `PROJECT-SUBPROJECT.zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that.

Although downloading the entire zip file is time consuming, it will make processing the individual files much more efficient and the code is less likely to break due to interruption in connectivity.


In [7]:
CHUNK = 16 * 1024
for project in tqdm.tqdm(projects):
    url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    r = requests.get(url, verify=False)
    if r.status_code == 200:
        print("Downloading " + url + " saving as " + file)
        with open(file, 'wb') as f:
            for c in r.iter_content(chunk_size=CHUNK):
                f.write(c)
        print('Waiting 3 seconds in order not to overload the ORACC server')
        time.sleep(3)
    else:
        print(url + " does not exist.")

  0%|          | 0/139 [00:00<?, ?it/s]c:\Users\valek\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'build-oracc.museum.upenn.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Waiting 3 seconds in order not to overload the ORACC server


  1%|          | 1/139 [00:04<10:39,  4.63s/it]c:\Users\valek\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'build-oracc.museum.upenn.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Waiting 3 seconds in order not to overload the ORACC server


  1%|▏         | 2/139 [00:09<10:36,  4.65s/it]c:\Users\valek\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'build-oracc.museum.upenn.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

 99%|█████████▊| 137/139 [10:33<00:11,  5.76s/it]c:\Users\valek\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'build-oracc.museum.upenn.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Waiting 3 seconds in order not to overload the ORACC server


 99%|█████████▉| 138/139 [10:37<00:05,  5.30s/it]c:\Users\valek\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'build-oracc.museum.upenn.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Waiting 3 seconds in order not to overload the ORACC server


100%|██████████| 139/139 [10:41<00:00,  4.61s/it]


# Extracting ZIP files

TODO: add function description

In [8]:
def extract_and_delete_zip():
    zipped_projects = os.listdir(ZIP_PATH)
    for z_file in zipped_projects:
        if z_file[-4:] == '.zip':
            with zipfile.ZipFile(os.path.join(ZIP_PATH, z_file), 'r') as zip_ref:
                zip_ref.extractall(EXTRACT_PATH)
    
            os.remove(os.path.join(ZIP_PATH, z_file))
    
            print(f"File {z_file} has been extracted to folder projectsdata and deleted.")

In [9]:
extract_and_delete_zip()

File adsd-adart1.zip has been extracted to folder projectsdata and deleted.
File adsd-adart2.zip has been extracted to folder projectsdata and deleted.
File adsd-adart3.zip has been extracted to folder projectsdata and deleted.
File adsd-adart5.zip has been extracted to folder projectsdata and deleted.
File adsd-adart6.zip has been extracted to folder projectsdata and deleted.
File adsd.zip has been extracted to folder projectsdata and deleted.
File aemw-alalakh-idrimi.zip has been extracted to folder projectsdata and deleted.
File aemw-amarna.zip has been extracted to folder projectsdata and deleted.
File akklove.zip has been extracted to folder projectsdata and deleted.
File amgg.zip has been extracted to folder projectsdata and deleted.
File ario.zip has been extracted to folder projectsdata and deleted.
File armep.zip has been extracted to folder projectsdata and deleted.
File arrim.zip has been extracted to folder projectsdata and deleted.
File asbp-ninmed.zip has been extracted t